In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import base64
import cv2
import os
import io
from IPython.display import HTML, display
import cv2
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [11]:
def img_to_html(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    _, buffer = cv2.imencode('.png', img_rgb)
    base64_img = base64.b64encode(buffer).decode('utf-8')
    return f'<img src="data:image/png;base64,{base64_img}" width="80" height="80" />'

In [12]:
def extrair_histogramas(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist_gray = cv2.calcHist([gray], [0], None, [256], [0, 256])
    hist_gray = cv2.normalize(hist_gray, hist_gray).flatten()

    hist_b = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([image], [2], None, [256], [0, 256])

    hist_rgb = np.concatenate([
        cv2.normalize(hist_b, hist_b).flatten(),
        cv2.normalize(hist_g, hist_g).flatten(),
        cv2.normalize(hist_r, hist_r).flatten()
    ])
    return np.concatenate([hist_gray, hist_rgb])

In [13]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif', 'bmp', 'tiff'}

def allowed_file(filename):
    return '.' in filename and \
        filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [14]:
path = "../dataset"
all_images_paths = []

def find_files(path):
    for name in os.listdir(path):
        full_path = os.path.join(path, name)
        if os.path.isdir(full_path):
            find_files(full_path)
        elif os.path.isfile(full_path) and allowed_file(name):
            all_images_paths.append(full_path)

find_files(path)

In [ ]:
all_images = [cv2.imread(name) for name in all_images_paths]
X = np.array([extrair_histogramas(img) for img in all_images])

query_img = cv2.imread("../dataset/Apple/9_100.jpg")
query = extrair_histogramas(query_img).reshape(1, -1)

knn = NearestNeighbors(n_neighbors=len(all_images_paths), metric="euclidean")
knn.fit(X)

distances, indices = knn.kneighbors(query)

In [ ]:
results = []
for rank, idx in enumerate(indices[0]):
    image_name = all_images_paths[idx]
    distance = round(float(distances[0][rank]), 5)
    similarity = round(float(1 / (1 + distance)), 2)
    image = img_to_html(all_images[idx])
    results.append({
        "Rank": rank + 1,
        "Nome da Imagem":image_name,
        "Distância": distance,
        'Similaridade': similarity,
        "Imagem": image
    })

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="Distância", ascending=True).reset_index(drop=True)
pd.set_option('display.max_colwidth', None)
# display(HTML(df_results.to_html(escape=False)))

df_results[df_results['Rank'] == 1]['Nome da Imagem']


0    ../dataset/Apple/9_100.jpg
Name: Nome da Imagem, dtype: object